# Ultrasonic Distance Ranger


## Particle CLI

### Installation

In [1]:
%%capture
!bash <( curl -sL https://particle.io/install-cli )

# path to the particle cli. May be environment dependent.
particle_cli = "/root/bin/particle"

### Utility functions

In [2]:
import re
import subprocess

# regular expression to strip ansi control characters
ansi = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

# decode byte string and strip ansi control characters
def decode_bytes(byte_string):
    if isinstance(byte_string, bytes):
        result = byte_string.decode("utf-8")
    return ansi.sub("", result)

# streamline call to the particle-cli
def particle(args):
    process = subprocess.run(["/root/bin/particle"] + args,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE)
    process.stdout = decode_bytes(process.stdout)
    process.stderr = decode_bytes(process.stderr)
    return process

### Login to Particle

In [3]:
import getpass

# prompt for username and password
username = getpass.getpass(prompt="Username: ")
password = getpass.getpass(prompt="Password: ")

# attempt login
output = particle(["login", "--username", username, "--password", password])

# report results
if output.returncode:
    print(f"Return code = {output.returncode}")
    print(output.stderr)
else:
    print(output.stdout)

Username: ··········
Password: ··········
> Successfully completed login!



### Select a device

The following cell downloads a list of all user devices and creates a list of device names. Here we choose the first name in the list for the rest of this notebook. If this is not the device to be used, then modify this cell accordingly.

In [4]:
devices = [line.split()[0] for line in particle(["list"]).stdout.splitlines()]
device_name = devices[0]
print(particle(["list", device_name]).stdout)

jck_argon_01 [e00fce68eaceb1faa7cf7193] (Argon) is online



## Project Requirements

Create a simple rangefinder that reports distance, measured in centimeters, to a real-time display.



## Project Hardware

### Grove Ultrasonic Ranger

[SeeedStudio description](https://wiki.seeedstudio.com/Grove-Ultrasonic_Ranger/)

[Github](https://github.com/Seeed-Studio/Seeed_Arduino_UltrasonicRanger)

![](https://files.seeedstudio.com/wiki/Grove_Ultrasonic_Ranger/img/Ultrasonic.jpg)

### Grove 4 Digit Display

## Implementation

### Create Project

In [5]:
print(particle(["project", "create", "--name", "myproject", "."]).stdout)

Initializing project in directory myproject...
> A new project has been initialized in directory myproject



### Change working directory

The Particle CLI assumes one is working in the top  project directory.

In [6]:
%cd /content/myproject

/content/myproject


### Add relevant libraries

In [11]:
print(particle(["library", "search", "Ultrasonic"]).stdout)

> Found 2 libraries matching Ultrasonic
Grove-Ultrasonic-Ranger 1.0.2 16133 Particle Xenon library for controlling Grove Ultrasonic Ranger, using gennal I/O communication.
PatriotUltrasonic 2.0.0 128 Extend Patriot IoT to support ultrasonic proximity detectors.



In [30]:
print(particle(["library", "add", "Grove-Ultrasonic-Ranger"]).stdout)
print(particle(["library", "add", "Grove_4Digit_Display"]).stdout)

Checking library Grove-Ultrasonic-Ranger...
Installing library Grove-Ultrasonic-Ranger 1.0.2 to /content/myproject/lib/Grove-Ultrasonic-Ranger ...
Library Grove-Ultrasonic-Ranger 1.0.2 installed.

> Library Grove_4Digit_Display 1.0.2 has been added to the project.
> To get started using this library, run particle library view Grove_4Digit_Display to view the library documentation and sources.



### Create source file

In [48]:
%%writefile src/myproject.ino

/* pin assignments */
#define PIN_CLK D2         /* display clock */
#define PIN_DIO D3         /* display data */
#define PIN_ULTRASONIC D4  /* ultrasonic ranger */

/* display parameters */
#define DIGITS 4   /* display digits */

#include "Grove-Ultrasonic-Ranger.h"
#include "Grove_4Digit_Display.h"

/* hardware objects */
Ultrasonic ultrasonic(PIN_ULTRASONIC);
TM1637 tm1637(PIN_CLK, PIN_DIO);

long int count =  0;

void setup() {
    /* setup display */
    tm1637.init();
    tm1637.set(BRIGHT_TYPICAL);
    tm1637.point(POINT_OFF);
}

void loop() {
    display(ultrasonic.MeasureInCentimeters());
    delay(100);
}

void display(unsigned int number) {
    for (int i = 0; i < 4; i++) {
        int digit = DIGITS - 1 - i;
        tm1637.display(digit, number % 10);
        number /= 10;
    }
}

Overwriting src/myproject.ino


### Compiling

In [49]:
print(particle(["compile", "argon", "--saveTo", "myproject.bin"]).stdout)


Compiling code for argon

Including:
    lib/Grove-Ultrasonic-Ranger/src/Grove-Ultrasonic-Ranger.h
    lib/Grove-Ultrasonic-Ranger/examples/UltraDisOnSeeedSerialLcd/UltraDisOnSeeedSerialLcd.ino
    lib/Grove-Ultrasonic-Ranger/examples/UltrasonicDisplayOnTerm/UltrasonicDisplayOnTerm.ino
    src/myproject.ino
    lib/Grove-Ultrasonic-Ranger/src/Grove-Ultrasonic-Ranger.cpp
    project.properties

attempting to compile firmware
downloading binary from: /v1/binaries/5f946cd8186082895481df8a
saving to: myproject.bin
Memory use:
   text	   data	    bss	    dec	    hex	filename
   6412	    108	   1072	   7592	   1da8	/workspace/target/workspace.elf

Compile succeeded.
Saved firmware to: /content/myproject/myproject.bin



### Flash firmware

In [41]:
print(particle(["flash", device_name, "myproject.bin"]).stdout)

Including:
    myproject.bin

attempting to flash firmware to your device jck_argon_01
Flash device OK: Update started

Flash success!

